# 导入依赖库

In [79]:
import json
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from utils import get_ret_mat, get_data_dict


# 参数配置

In [80]:
n_top = 10  # 每周选取前 n_top 只股票
score_path = 'data/score.csv'
reb_dict_path = 'data/reb_dict.json'

# 工具函数

In [81]:
def compute_risk_parity_weights(cov, max_iter=1000, tol=1e-8):
    """使用SLSQP求解等风险贡献权重"""
    n = cov.shape[0]

    # objective: sum of squared deviations of risk contributions to equal target
    def objective(w):
        # portfolio risk contributions
        z = cov.dot(w)
        rc = w * z
        target = np.sum(rc) / n
        return np.sum((rc - target) ** 2)

    # gradient of the objective
    def gradient(w):
        z = cov.dot(w)
        rc = w * z
        target = np.sum(rc) / n
        g = rc - target  # vector of deviations

        # derivative of target wrt w: 2/n * (Sigma w)
        d_target = 2.0 * z / n

        # compute gradient: grad_i = 2 * sum_j g_j * (d_rc_j/d_w_i - d_target_i)
        # where d_rc_j/d_w_i = delta_{ij} z_j + w_j * cov[j, i]
        # and d_target_i same for all j
        # we'll compute elementwise
        grad = np.zeros_like(w)
        for i in range(n):
            # compute d_rc_j/d_w_i for all j
            d_rc = np.zeros(n)
            # term when j == i: z_i
            d_rc[i] = z[i]
            # term for all j: w_j * cov[j, i]
            d_rc += w * cov[:, i]
            # derivative contributions
            grad[i] = 2.0 * np.dot(g, d_rc - d_target[i])
        return grad

    # constraints: sum(w) == 1
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0,
                    'jac': lambda w: np.ones_like(w)})
    # bounds: w >= 0
    bounds = tuple((1/(2*n), 2/n) for _ in range(n))

    # initial guess: equal weights
    w0 = np.ones(n) / n

    result = minimize(
        objective,
        w0,
        method='SLSQP',
        jac=gradient,
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': max_iter, 'ftol': tol, 'disp': False}
    )

    if not result.success:
        raise RuntimeError(f"Risk parity optimization failed: {result.message}")

    return result.x

In [82]:
def get_reb_dict(score_df, n_top, risk_parity=False, lookback_window=60):
    """
    输入：
      - score_df: D, taFrame, index 为日期, columns 为标的代码, values 为得分
      - n_top: 每次选股数量
      - lookback_window: 协方差估计窗口长度
    返回：
      - dict of {rebalance_date: {code: weight, ...}, ...}
    """
    sel_weights = {}
    ret_df = get_ret_mat(data_dict=get_data_dict(start='2024-06-01')) * 100
    dates =  ret_df.index
    week_ends = score_df.iloc[:, 0].resample('W-FRI').apply(lambda x: x.index[-1]).tolist()  # 每周最后一个交易日
    for dt in week_ends:
        # 获取当日得分并选前 n_top
        scores = score_df.loc[dt].dropna()
        top_codes = scores.nlargest(n_top).index.tolist()

        # 等权配置
        if not risk_parity:
          sel_weights[dt.strftime('%Y-%m-%d')] = dict(zip(top_codes, np.ones(n_top)/n_top))
        
        # 风险平价配置
        else:
            # 选取前 lookback_window 个交易日, 计算协方差矩阵
            end = dt
            loc = dates.get_loc(dt)
            start_loc = max(0, loc - lookback_window)
            start = dates[start_loc]
            rets = ret_df.loc[start:end, top_codes]
            cov = rets.cov().values

            # 计算风险平价权重
            w = compute_risk_parity_weights(cov)
            sel_weights[dt.strftime('%Y-%m-%d')] = dict(zip(top_codes, w))

    return sel_weights

# 求解风险平价配置

In [83]:
# 获取选股及权重结果
score_df = pd.read_csv(score_path, index_col=0, parse_dates=True)
reb_dict = get_reb_dict(score_df, n_top, risk_parity=False)

In [84]:
# 存储字典
with open(reb_dict_path, 'w') as f:
    json.dump(reb_dict, f)